#### Importing packages

In [ ]:
# Load packages 

import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymongo
import seaborn as sns
import uuid

from collections import Counter
from datetime import datetime
from dateutil import parser


## Overview

1. <strong>Load annotations for a task </strong>


2. <strong>Calculate stats related to this task, e.g. number of workers, time for annotation, etc. </strong>


3. <strong>Apply majority voting and update database </strong> 

Set the following variables first:


In [ ]:
# Set the task to be postprocessed: "table_relevance", "claim_adjustment", "verification"
task = ""

# Load file with annotations into pandas.DataFrame OR if annotations saved in DB, create client and load data, example below 
df = pd.DataFrame()

# List of banned workers, if any (.json file)
path_banned_workers = ''


--------

In [ ]:
# Load annotation data from MongoDB 

with open(path_database_config,'r') as f:
    mongodb_credentials = json.load(f)

db_client = pymongo.MongoClient(mongodb_credentials["connection_string"])
db = db_client[database_name]

hit_result_collection = db.hit_results
cursor = hit_result_collection.find({"type": task})
df = pd.DataFrame(list(cursor))

print(f"Number of samples in 'hit_result_collection' is {len(df_unfiltered)}")


### Create dataframe

In [ ]:
# load list of banned workers to exclude them
with open(path_banned_workers,'r') as f:
    banlist = json.load(f)

# load list of rejected assignments to exclude them (if any exist)
rejected_assignments = pd.read_excel("filled_answer_df.xlsx")
rejected_assignments = list(rejected_assignments[rejected_assignments["reject"]==1].assignment_id)

print(f"Number of banned workers is {len(banlist)}.")
print(f"Number of rejected assignments is {len(rejected_assignments)}.")


In [ ]:
cols = list(df.columns)
cols.remove("language")
cols.remove("type")
cols.remove("timestamp")
cols.remove("answers")
cols.extend(['worker_id', 'outputs', 'times', 'events', 'feedback'])

annotations_df = pd.DataFrame(columns = cols)
counter_skip = 0
index = 0
for i, row in df.iterrows(): 
    if task != "claim_generation" and (type(row["answers"])!=list or len(row["answers"]) < 3): # we only consider df entries with completed assignments 3/3
        counter_skip += 1
        continue 
    
    for worker_answer in row["answers"]:
        if worker_answer["worker_id"] in banlist or worker_answer["assignment_id"] in rejected_assignments:
            print("Skipped because worker is banned or assignment has been rejected.")
            counter_skip += 1
            continue
        
        row["worker_id"] = worker_answer["worker_id"]
        annotations_df.at[index, "_id"] = row["_id"]
        annotations_df.at[index, "batch_id"] = row["batch_id"]
        annotations_df.at[index, "references"] = row["references"]
        annotations_df.at[index, "taskSet_id"] = row["taskSet_id"]
        annotations_df.at[index, "hit"] = row["hit"]
        
        annotations_df.at[index, "worker_id"] = worker_answer["worker_id"]
        annotations_df.at[index, "outputs"] = worker_answer["values"]["outputs"]
        annotations_df.at[index, "times"] = worker_answer["values"]["times"]
        annotations_df.at[index, "events"] = worker_answer["values"]["events"]
        annotations_df.at[index, "feedback"] = worker_answer["values"]["feedback"]
        index += 1

print(len(annotations_df))
counter_skip


In [ ]:
# | worker | claim id | label |

if task in ["table_annotation", "adjusted_claim_annotation"]:
    output_df = pd.DataFrame(columns=["HIT_id", "batch_id", "taskSet_id", "worker_id", "claim_id", "claim", "table",
                                      "label", "header", "events"])

    index = 0
    for i, row in annotations_df.iterrows():
        for j in range(len(row["references"])):
            if row["references"][j]["g_id"]!=-1:
                continue

            output_df.at[index, "HIT_id"] = row["_id"]
            output_df.at[index, "batch_id"] = row["batch_id"]
            output_df.at[index, "taskSet_id"] = row["taskSet_id"]
            output_df.at[index, "worker_id"] = row["worker_id"]

            output_df.at[index, "claim_id"] = row["references"][j]["claim_db_id"]
            output_df.at[index, "claim"] = row["references"][j]["claim"]
            output_df.at[index, "table"] = row["references"][j]["table"]

            output_df.at[index, "label"] = row["outputs"][j]["label"]
            output_df.at[index, "header"] = row["outputs"][j]["header"]
            output_df.at[index, "events"] = row["events"][j]
            index += 1

    print(len(output_df))
    
elif task == "claim_generation": 
    
    output_df = pd.DataFrame(columns=["HIT_id", "batch_id", "taskSet_id", "worker_id", "claim_id", "claim", "table",
                                      "label", "adjusted_claim", "events"])
    index = 0
    for i, row in annotations_df.iterrows():
        for j in range(len(row["references"])):

            output_df.at[index, "HIT_id"] = row["_id"]
            output_df.at[index, "batch_id"] = row["batch_id"]
            output_df.at[index, "taskSet_id"] = row["taskSet_id"]
            output_df.at[index, "worker_id"] = row["worker_id"]

            output_df.at[index, "claim_id"] = row["references"][j]["claim_db_id"]
            output_df.at[index, "claim"] = row["references"][j]["claim"]
            output_df.at[index, "table"] = row["references"][j]["table"]

            output_df.at[index, "adjusted_claim"] = row["outputs"][j]["adjusted_claim"]
            output_df.at[index, "label"] = row["outputs"][j]["label"]
            output_df.at[index, "events"] = row["events"][j]
            index += 1

    print(len(output_df))
    

In [ ]:
stats_df = output_df[['claim', 'table']].copy()
print(len(set(stats_df["claim"])))

urls = [entry["url"] for entry in stats_df["table"]]
print(len(set(urls)))

html_table = [entry["html_table"] for entry in stats_df["table"]]
print(len(set(html_table)))


In [ ]:
x = []

for index, row in output_df.iterrows():
    if row["label"] not in [0, 1]: 
        continue
    elif row["claim"].strip() not in x:
        x.append(row["claim"].strip())

for _x in x: 
    print(f"* {_x}")
#     print()

### Worker analysis

In [ ]:
worker_list = output_df["worker_id"]
print(f"{len(list(set(worker_list)))} unique workers worked on the {len(annotations_df)} tasks.")

Counter(worker_list).most_common()


### Average annotation time

In [ ]:
def calc_annotation_time(events: list): 
    first_timestamp = parser.isoparse(events[1]['timestamp'])
    last_timestamp = parser.isoparse(events[len(events)-1]['timestamp'])

    duration = last_timestamp - first_timestamp
    minutes = (duration.seconds//60)
    
    return duration.seconds, minutes
    

In [ ]:
duration_min = []
duration_sec = []
worker_list = []

for index, row in annotations_df.iterrows(): 
    worker_id = row['worker_id']
    if worker_id in worker_list:
#         Uncomment rows below if we want to know time of non-first time workers
        dur_sec, dur_min = calc_annotation_time(row["events"])
        duration_sec.append(dur_sec)
        duration_min.append(dur_min)
#         continue
    else:
        worker_list.append(worker_id)
        dur_sec, dur_min = calc_annotation_time(row["events"])
        duration_sec.append(dur_sec)
        duration_min.append(dur_min)
        
        
# print boxplot of time duration for task 

fig, ax = plt.subplots(1,2, figsize=(16, 4))
sns.boxplot(x=duration_min, ax = ax[0])
sns.boxplot(x=duration_sec, ax = ax[1])

ax[0].set_title('Time spent on one HIT (first time workers)')
ax[1].set_title('Time spent on one HIT (first time workers)')
ax[0].set(xlabel='time in minutes')
ax[1].set(xlabel='time in seconds')


print(pd.DataFrame(duration_min).describe())


### Label distribution

In [ ]:
len(output_df)

In [ ]:
df = pd.DataFrame({'label': output_df["label"]})
ax = sns.barplot(x=df.label.value_counts().index, y=df.label.value_counts())

s = 0
for p in ax.patches:
    s+= p.get_height()

for p in ax.patches: 
    ax.text(p.get_x() + p.get_width()/2.,
            p.get_height(),
            '{}'.format(int(p.get_height())), 
            fontsize=12,
            color='black',
            ha='center',
            va='bottom')

ax.set_title('Distribution of labels')
ax.set(xlabel='0 = supports | 1 = refutes | 2 = related but NEI | 3 = unrelated ', ylabel='annotations')
# ax.set(xlabel='0 = supported | 1 = refuted ', ylabel='adjusted claims')

plt.show()




### [TABLE ANNOTATION TASK] Majority voting 

In [ ]:
def majority(votes):
    votes = [v for v in votes if v != -1]
    if collections.Counter(votes).most_common(1)[0][1] == 1: 
        return -1
    else:
        return collections.Counter(votes).most_common(1)[0][0]


In [ ]:
pre_label_df = output_df[['HIT_id', 'claim_id','label','header']].copy()
pre_label_df['label'] = pre_label_df['label'].apply(lambda x : [x])
pre_label_df['header'] = pre_label_df['header'].apply(lambda x : [x] if x==-1 else x)
pre_label_df.head()


In [ ]:
label_df = pre_label_df.groupby(['claim_id', 'HIT_id']).agg({'label': "sum"}) 
label_df.head()


In [ ]:
# label_df = pre_label_df.groupby(['claim_id', 'HIT_id']).agg('sum') 
# label_df = pre_label_df.groupby(['majority']).agg('sum')

label_df = label_df.dropna()

print(f"Length before filtering for claims with three assignments: {len(label_df)}")
label_df = pd.DataFrame([row for index, row in label_df.iterrows() if len(row["label"])==3])
label_df['majority'] = label_df['label'].apply(majority)

label_df = label_df.reset_index()
print(len(label_df))
label_df.head(10)


In [ ]:
df = pd.DataFrame({'label': label_df["majority"]})
ax = sns.barplot(x=df.label.value_counts().index, y=df.label.value_counts())

s = 0
for p in ax.patches:
    s+= p.get_height()

for p in ax.patches: 
    ax.text(p.get_x() + p.get_width()/2.,
            p.get_height(),
            '{}'.format(int(p.get_height())), 
            fontsize=12,
            color='black',
            ha='center',
            va='bottom')

ax.set_title('Distribution of labels')
ax.set(xlabel='0 = supports | 1 = refutes | 2 = related but NEI | 3 = unrelated ', ylabel='claim-table pairs')
plt.show()


In [ ]:
label_df.head()

### Update majority voting label in final_dataset in DB (=> needed for claim generation)

In [ ]:
final_ds_col = db.final_dataset

cursor =  final_ds_col.find({})
df = pd.DataFrame(list(cursor)) 
print(f"Length of test set: {len(df)}")
df.head(3)


In [ ]:
# how many (claim, relevant_table) pairs we have in final_ds

temp = [len(x) for x in df["table_relevant"] if x and type(x) == list]
sum(temp) # total


In [ ]:
# Extract from output_df all entries with majority voting "2" (="relevant")

x = []
for claim_id, hit_id in list(label_df[label_df["majority"]==2]["index"]):
    x.append(output_df[(output_df["claim_id"]==claim_id) & (output_df["HIT_id"]==hit_id)].iloc[0,:])

output_df_majority = pd.DataFrame(x)
print(len(output_df_majority))
output_df_majority.head(1)


In [ ]:
# update final_dataset collection in MongoDB 
# Add tables to column "table_relevant" in final_dataset

counter = 0
for index, row in df.iterrows(): # iterate over final_dataset
    _id = str(row["_id"])
    
    for i, r in output_df_majority.iterrows(): # iterate over claims with relevant tables (accord. to majority voting)
        annotated_table = r["table"]
        change = 0
        
        if _id != r["claim_id"]:
            continue 
            
        temp_list = row["table_relevant"] if type(row["table_relevant"]) == list else []
        relevant_tables = []
        for relevant_t in temp_list:
            if "_id" in relevant_t:
                relevant_t["id"] = relevant_t["_id"]
                del relevant_t["_id"]
                
            relevant_tables.append(relevant_t)
        
        if "id" in annotated_table: 
            if annotated_table["id"] not in [t["id"] for t in relevant_tables]:
                relevant_tables.append(annotated_table)
                change = 1
                
        elif "_id" in annotated_table: 
            if annotated_table["_id"] not in [t["id"] for t in relevant_tables]:
                annotated_table["id"] = annotated_table["_id"].copy()
                del annotated_table["_id"]
                relevant_tables.append(annotated_table)
                change = 1

        else: # find matching _id for annotated_table first
            for table in row["tables"]: 

                if table["rows_list"] == annotated_table["rows_list"]:
                    not_added_before = False
                    for tab in relevant_tables:
                        if annotated_table["rows_list"] == tab["rows_list"]: 
                            not_added_before = True

                    if not_added_before:
                        annotated_table["id"] = table["id"]
                        relevant_tables.append(annotated_table)
                        change = 1

        if change:
            counter += 1
            final_ds_col.update_one({'_id': row["_id"]},
                                    {'$set': {'table_relevant': relevant_tables}})

counter
        

### Update majority voting in claim_generation dataset (=> after adjusted claim annotation)

In [ ]:
claim_col = db.claim_generation
cursor =  claim_col.find({})
df = pd.DataFrame(list(cursor)) 
print(f"Length of data set: {len(df)}")
df.head(3)


In [ ]:
df['_id_str'] = df['_id'].astype('str')
no_entries = []

for index, row in label_df.iterrows(): 
    _id = row["index"][0]
    row_id = df.loc[df['_id_str'] == _id]["_id"]
    
    if len(row_id.values) == 0:
        no_entries.append(row["index"])
        continue
    
#         Uncomment to update claim_generation collection in MongoDB
#     claim_col.update_one({'_id': row_id.values[0]}, 
#                          {
#                              '$set': {'majority_label': str(row["majority"]), 
#                                       'majority_header': row["header"]}
#                          })

print(len(no_entries))


In [ ]:
no_entries[0]

In [ ]:
output_df.head(1)

In [ ]:
# todo enter for ids in "no_entries" list the corrsponding claim table pair to the collection "claim_generation" in MongoDB

for claim_id, hit_id in no_entries: 
    row = output_df.loc[(output_df['claim_id'] == claim_id) && (output_df['HIT_id'] == hit_id)]
    


### Header rationales 

In [ ]:

label_list = [len(entry) for entry in output_df["header"] if (entry!=-1 and entry!=[])]
df = pd.DataFrame({'label': label_list})
# sns.histplot(data=label_list)

ax = sns.barplot(x=df.label.value_counts().index, y=df.label.value_counts())

s = 0
for p in ax.patches:
    s+= p.get_height()

for p in ax.patches: 
    ax.text(p.get_x() + p.get_width()/2.,
            p.get_height(),
            '{}'.format(int(p.get_height())), 
            fontsize=11,
            color='black',
            ha='center',
            va='bottom')
    
ax.set_title('Number of column rationales selected')
ax.set(ylabel='count')

plt.show()


In [ ]:
workers = []
for index, row in annotations_df.iterrows():
    for output in row["outputs"]: 
        if output['label']==0:
            workers.append(row["worker_id"])
            
print("Workers + count of claims they selected header rationales for:")
Counter(workers).most_common()

### [CLAIM GENERATION TASK] Adjusted Claims

In [ ]:
add_adjusted_claims = True
claim_generation_col = db.claim_generation
counter = 0

if add_adjusted_claims:
    for index, row in output_df.iterrows(): 
        if list(claim_generation_col.find({'claim': row["adjusted_claim"]})) == []:
            counter += 1
            claim_generation_col.insert_one({"_id": uuid.uuid4(), 
                                             "claim": row["adjusted_claim"],
                                             "table": row["table"],
                                             "label": row["label"],
                                             "initial_claim_id": row["claim_id"],
                                             "initial_claim": row["claim"]})
print(counter)


### Feedback

In [ ]:
[entry for entry in annotations_df["feedback"] if entry!=None]

### Some further analysis

In [ ]:
# label_list = [str(entry["label"]) for annotation in annotations_df["outputs"] for entry in annotation]
df = pd.DataFrame({'label': output_df["label"]})


In [ ]:
output_df

In [ ]:
related_pairs = {}
for index, row in output_df.iterrows():
    if row["label"] == 2: 
        related_pairs[row["claim"]] = [row["worker_id"]]

len(related_pairs)


In [ ]:
list(related_pairs.keys())

In [ ]:
db_client.close()

#### Add output_df_majority to final dataset to balance NEI class

In [ ]:
import copy
import jsonlines

from sklearn.model_selection import train_test_split


In [ ]:
output_df_majority.head(3)

In [ ]:
# templates used for dataset creation

entry_template = {
    "_id": "", 
    "claim": "", 
    "label": "", 
    "header_rationale": "", 
    "table": "",
    "initial_claim": ""
}

table_template = {
    'website': "",
    'website_title': "",
    'caption': "",
    'header_horizontal': "",
    'header_vertical': "",
    'rows': "",
    'html_code': ""
}


In [ ]:
# Load dataset

dataset = []
with jsonlines.open('data/dataset.jsonl') as reader:
    for line in reader: 
        dataset.append(line)

len(dataset)

In [ ]:
list_of_claims = list(pd.DataFrame(dataset)["claim"])
len(list_of_claims)

In [ ]:
result_dict=[]

#removing the duplicate entry
for i in range(len(Langlist)): 
    if Langlist[i] not in Langlist[i + 1:]: 
        result_dict.append(Langlist[i]) 


In [ ]:
# create dataset using templates

for index, row in output_df_majority.iterrows():
    if row["claim"] in list_of_claims: 
        continue
    else: 
        list_of_claims.append(row["claim"])
    
    table = copy.deepcopy(table_template)
    table["website"] = row["table"]["url"]
    table["website_title"] = row["table"]["title"]
    table["caption"] = row["table"]["caption"]
    table["header_horizontal"] = row["table"]["header_horizontal"]
    table["header_vertical"] = row["table"]["header_vertical"]
    table["rows"] = row["table"]["rows_list"]
    table["html_code"] = row["table"]["html_table"]
    
    entry = copy.deepcopy(entry_template)
    entry["table"] = table
    entry["_id"] = str(row["claim_id"])
    entry["claim"] = row["claim"]
    entry["label"] = "NOT ENOUGH INFO"
    entry["header_rationale"] = row["header"]
    entry["initial_claim"] = row["claim"]
    
    dataset.append(entry)

print(len(dataset))


In [ ]:
trainset, testset = train_test_split(dataset.copy(), test_size=0.2)  
testset, evalset = train_test_split(testset.copy(), test_size=0.5)  

print(len(trainset))
print(len(evalset))
print(len(testset))


In [ ]:
# Load data

# save dataset in jsonl file 

# with jsonlines.open('data/dataset_balanced.jsonl', mode='w') as writer:
#     writer.write_all(dataset)

# with jsonlines.open('data/trainset.jsonl', mode='w') as writer:
#     writer.write_all(trainset)

# with jsonlines.open('data/evalset.jsonl', mode='w') as writer:
#     writer.write_all(evalset)

# with jsonlines.open('data/testset.jsonl', mode='w') as writer:
#     writer.write_all(testset)
